In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np

In [ ]:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

### Load Registration List

In [ ]:
df_registered = pd.read_csv('../data/state_of_michigan_foia/March Entire State Reg List 2025.csv', encoding='ISO-8859-1')
df_registered.sample()

### Join Census County Codes

In [ ]:
def mapCountyCodes(mich_code):
    df_counties = pd.read_csv('../data/custom_data/county_code_mapping.csv')
    df_census_code = df_counties[df_counties['Michigan County Code'] == mich_code]
    census_code = df_census_code['Census County Code'].values[0]
    return census_code

In [ ]:
# THIS WILL TAKE A VERY LONG TIME.
df_registered['WARD'] = df_registered['WARD'].fillna(0)
df_registered['WARD'] = df_registered['WARD'].astype(int)

df_registered['Census County Code'] = df_registered['COUNTY_CODE'].apply(mapCountyCodes)

df_registered['standardized_id_num'] = df_registered.apply(lambda row: str(row['Census County Code']) + str(row['JURISDICTION_CODE']).zfill(5) + str(row['PRECINCT']), axis=1)
df_registered['standardized_id_num'] = df_registered['standardized_id_num'].astype(str).str.zfill(13)

In [ ]:
df_registered.to_csv('../data/custom_data/registered_voters.csv', index=False)

### Registration Counts per Precinct

In [ ]:
df_unique = df_registered['standardized_id_num'].value_counts().reset_index()
df_unique.columns = ['standardized_id_num', 'count']
df_unique.head()

In [ ]:
df_unique.to_csv('../data/custom_data/registered_voters_count.csv', index=False)

### Load Voter History

In [ ]:
hist1 = pd.read_csv('../data/state_of_michigan_foia/March Entire State Voter History 2025 Part 1.csv')
hist2 = pd.read_csv('../data/state_of_michigan_foia/March Entire State Voter History 2025 Part 2.csv')

### Join Registration List and Voter History

In [ ]:
overlap_cols = df_registered.columns.intersection(hist1.columns).difference(['VOTER_IDENTIFICATION_NUMBER'])  # keep 'id' for merging

hist1 = hist1.drop(columns=overlap_cols)
hist1 = pd.merge(hist1, df_registered, on='VOTER_IDENTIFICATION_NUMBER', how='inner')

In [ ]:
overlap_cols = df_registered.columns.intersection(hist2.columns).difference(['VOTER_IDENTIFICATION_NUMBER'])  # keep 'id' for merging

hist2 = hist2.drop(columns=overlap_cols)
hist2 = pd.merge(hist2, df_registered, on='VOTER_IDENTIFICATION_NUMBER', how='inner')

hist2 = hist2.loc[:, ~hist2.columns.duplicated()]

In [ ]:
df_registration_history = pd.concat([hist1, hist2], axis=0)
df_registration_history = df_registration_history.drop(columns=['Unnamed: 8'])

In [ ]:
# df_registration_history.to_csv('../data/custom_data/df_registration_history.csv', index=False)

In [ ]:
df_registration_history.sample()

### Features
Kernel crashes

In [ ]:
df_registration_history = pd.read_csv('../data/custom_data/df_registration_history.csv')

In [ ]:
# # Run an apply() across several columns to update
# # several other columns.
# def applyElectionDate(row):
#     y = row['ELECTION_DATE'][:4]
#     m = row['ELECTION_DATE'][5:7]
#     d = row['ELECTION_DATE'][8:10]
#     return pd.Series({'ELECTION_YEAR': y, 'ELECTION_MONTH': m, 'ELECTION_DAY': d})
#
# # Crashes the kernel
# df_registration_history[['ELECTION_YEAR', 'ELECTION_MONTH', 'ELECTION_DAY']] = df_registration_history.apply(applyElectionDate, axis=1)

In [ ]:
# df_registration_history['ELECTION_YEAR'] = df_registration_history['ELECTION_DATE'].apply(lambda x: x[:4])
# df_registration_history['ELECTION_MONTH'] = df_registration_history['ELECTION_DATE'].apply(lambda x: x[5:7])
# df_registration_history['ELECTION_DAY'] = df_registration_history['ELECTION_DATE'].apply(lambda x: x[8:10])

### Tests

In [ ]:
df_registration_history[(df_registration_history['LAST_NAME'] == 'KETCHUM') & (df_registration_history['FIRST_NAME'] == 'NICHOLAS') & (df_registration_history['MIDDLE_NAME'] == 'RYAN')]